### Establishing a database connection and retrieving data

In [ ]:
driver = "com.microsoft.sqlserver.jdbc.SQLServerDriver"

database_host = "stocksdbserver.database.windows.net"
database_port = "1433" # update if you use a non-default port
database_name = "ANSQLBDD"
table = "stocks"
user = "ANid"
password = "salutCAVA145678+-"
url = f"jdbc:sqlserver://{database_host}:{database_port};database={database_name}"



In [ ]:
df = (
  spark.read
  .format("jdbc")
  .option("driver", driver)
  .option("url", url)
  .option("dbtable", table)
  .option("user", user)
  .option("password", password)
  .load()
)

### Data Exploration

In [ ]:
df.printSchema()
df.show(5)

+----------+------------------+------------------+------------------+------------------+----------+------------------+----------+
 date_| high_| low_| open_| close_| volume| adj_close|stock_name|
+----------+------------------+------------------+------------------+------------------+----------+------------------+----------+
2017-01-03|29.082500457763672|28.690000534057617|28.950000762939453|29.037500381469727|1.151276E8|27.277639389038086| APPLE|
2017-01-03| 789.6300048828125| 775.7999877929688| 778.8099975585938| 786.1400146484375| 1657300.0| 786.1400146484375| GOOGLE|
2017-01-03|117.83999633789062|115.51000213623047|116.02999877929688|116.86000061035156| 2.06639E7|116.86000061035156| FACEBOOK|
2017-01-03|44.066001892089844| 42.19200134277344| 42.97200012207031| 43.39799880981445| 2.96165E7| 43.39799880981445| TESLA|
2019-04-18| 66.0| 60.32099914550781| 65.0| 62.0| 2.57647E7| 62.0| ZOOM|
+----------+------------------+------------------+------------------+------------------+----------+------------------+----------+
only showing top 5 rows

In [ ]:
# Manage date_field type
from pyspark.sql.functions import to_date

df = df.withColumn("date_", to_date("date_"))
# This will convert the values in the "date_" column of df to the date data type, 
# and store the results in a new column named "date_".

#### 1. Return Rate

##### This implementation first filters the DataFrame df to keep only the rows with the specified stock and date range, using the filter method and the col function from pyspark.sql.functions. Then, it selects the initial and end values for the stock using filter and select, and computes the difference in days between the start and end dates using the datediff function from pyspark.sql.functions. Finally, it computes the return rate using the same formula as before.

In [ ]:
from pyspark.sql.functions import col, datediff

def get_return_rate(df, stock, start_date, end_date):
    '''Get the return rate for a specific stock and time interval'''

    temp = df.filter((col("stock_name") == stock) & (col("date_").between(start_date, end_date)))

    init_val = temp.filter(col("date_") == start_date).select("close_").first()[0]
    end_val = temp.filter(col("date_") == end_date).select("close_").first()[0]

    delta = datediff(end_date, start_date)

    result = ((end_val - init_val) / init_val * 100.0) / delta

    return result

res = get_return_rate(df, 'APPLE', '2017-01-03', '2017-02-03')


Out[4]: 0.24195594537491885

#### Moving Average

In [ ]:
from pyspark.sql.functions import col, avg, desc, lag

def calculate_moving_average(df, stock_name, start_date, end_date, num_points=5):
    '''Function that takes a DataFrame, a stock name, a start date, an end date,
    the number of points to consider for the moving average, and adds a new column
    to the DataFrame with the values of the calculated moving average.'''

    # Select the rows corresponding to the given stock and date range
    temp = df.filter((col("stock_name") == stock_name) & (col("date_").between(start_date, end_date)))

    # Create lag features
    window = Window.partitionBy("stock_name").orderBy(col("date_").desc())

    for i in range(1, num_points+1):
        temp = temp.withColumn(f"lag_{i}", lag(col("open_"), i).over(window))

    # Calculate the moving average
    avg_cols = [col(f"lag_{i}") for i in range(1, num_points+1)]
    avg_col = sum(avg_cols) / num_points

    # Create a new column to store the moving average
    result = temp.withColumn(f"moving_average_{num_points}", avg_col)

    return result

moving_average_df = calculate_moving_average(df, "APPLE", "2017-01-03", "2017-02-03", 5)
moving_average_df.show()


+----------+-----------------+-----------------+-----------------+-----------------+---------+-----------------+----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
 date_| high_| low_| open_| close_| volume| adj_close|stock_name| prev_open__1| prev_open__2| prev_open__3| prev_open__4| prev_open__5| moving_average_5|
+----------+-----------------+-----------------+-----------------+-----------------+---------+-----------------+----------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
2017-01-03| 758.760009765625|747.7000122070312|757.9199829101562|753.6699829101562|3521100.0|753.6699829101562| AMAZON| null| null| null| null| null| null|
2017-01-04|759.6799926757812|754.2000122070312|758.3900146484375|757.1799926757812|2510500.0|757.1799926757812| AMAZON|757.9199829101562| null| null| null| null| null|
2017-01-05|782.4000244140625| 760.260009765625|761.5499877929688|780.4500122070312|5830100.0|780.4500122070312| AMAZON|758.3900146484375|757.9199829101562| null| null| null| null|
2017-01-06|799.4400024414062| 778.47998046875|782.3599853515625| 795.989990234375|5986200.0| 795.989990234375| AMAZON|761.5499877929688|758.3900146484375|757.9199829101562| null| null| null|
2017-01-09| 801.77001953125| 791.77001953125| 798.0|796.9199829101562|3446100.0|796.9199829101562| AMAZON|782.3599853515625|761.5499877929688|758.3900146484375|757.9199829101562| null| null|
2017-01-10| 798.0|789.5399780273438|796.5999755859375|795.9000244140625|2558400.0|795.9000244140625| AMAZON| 798.0|782.3599853515625|761.5499877929688|758.3900146484375|757.9199829101562| 771.643994140625|
2017-01-11| 799.5| 789.510009765625|793.6599731445312| 799.02001953125|2992800.0| 799.02001953125| AMAZON|796.5999755859375| 798.0|782.3599853515625|761.5499877929688|758.3900146484375|779.3799926757813|
2017-01-12|814.1300048828125| 799.5|800.3099975585938|813.6400146484375|4873900.0|813.6400146484375| AMAZON|793.6599731445312|796.5999755859375| 798.0|782.3599853515625|761.5499877929688| 786.433984375|
2017-01-13|821.6500244140625|811.4000244140625|814.3200073242188|817.1400146484375|3791900.0|817.1400146484375| AMAZON|800.3099975585938|793.6599731445312|796.5999755859375| 798.0|782.3599853515625| 794.185986328125|
2017-01-17| 816.0|803.4400024414062|815.7000122070312| 809.719970703125|3670500.0| 809.719970703125| AMAZON|814.3200073242188|800.3099975585938|793.6599731445312|796.5999755859375| 798.0|800.5779907226563|
2017-01-18| 811.72998046875| 804.27001953125| 809.5| 807.47998046875|2354200.0| 807.47998046875| AMAZON|815.7000122070312|814.3200073242188|800.3099975585938|793.6599731445312|796.5999755859375|804.1179931640625|
2017-01-19| 813.510009765625|807.3200073242188| 810.0|809.0399780273438|2540800.0|809.0399780273438| AMAZON| 809.5|815.7000122070312|814.3200073242188|800.3099975585938|793.6599731445312| 806.697998046875|
2017-01-20| 816.02001953125| 806.260009765625| 815.280029296875|808.3300170898438|3376200.0|808.3300170898438| AMAZON| 810.0| 809.5|815.7000122070312|814.3200073242188|800.3099975585938|809.9660034179688|
2017-01-23| 818.5|805.0800170898438|806.7999877929688|817.8800048828125|2797500.0|817.8800048828125| AMAZON| 815.280029296875| 810.0| 809.5|815.7000122070312|814.3200073242188| 812.960009765625|
2017-01-24| 823.989990234375| 814.5| 822.0|822.4400024414062|2971700.0|822.4400024414062| AMAZON|806.7999877929688| 815.280029296875| 810.0| 809.5|815.7000122070312| 811.456005859375|
2017-01-25|837.4199829101562|825.2899780273438|825.7899780273438| 836.52001953125|3922600.0| 836.52001953125| AMAZON| 822.0|806.7999877929688| 815.280029296875| 810.0| 809.5|812.7160034179688|
2017-01-26|843.8400268554688| 833.0| 835.530029296875|839.1500244140625|3586300.0|839.1500244140625| AMAZON|825.7899780273438| 822.0|806.7999877929688| 815.280029296875| 810.0|815.9739990234375|
2017-01-27|839.7000122070312|829.4400024414062| 839.0| 835.7700195312

In [ ]:
moving_average_df.write.format("jdbc") \
                .option("url", url) \
                .option("dbtable", "MovingAvgDf") \
                .option("user", user) \
                .option("password", password) \
                .save()